In [1]:
import geopandas as gpd
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import urllib.request
from datetime import datetime

In [2]:
with urllib.request.urlopen("https://www.trusselltrust.org/get-help/find-a-foodbank/foodbank-search/?foodbank_s=all&callback=?") as url:
    data = json.loads(url.read().decode()[2:-2])

The Trussell Trust interactive map uses a JSON file with a list of dictionaries. Each distionary is associated with a small region and can consist of multiple foodbank centres. So each dictionary in the list has an additional nested dictionary with centre-specific information. Below is an attempt to extract some important features and create a simpler list of dictionaries corresponding to each unique foodbank distribution centre.

The centre geolocation and opening time information is contained in a third nested dictionary which should be extracted appropriately.

The Trussell Trust doesn't provide stock quantities for foodbanks but it does provide opening and closing times. My idea is to calculate weekly time spend open for each food bank, with more time spent open indicating higher risk of food insecurity for that region.

In [3]:
# Opening time information consists of dictionaries for each day, containing the opening and closing time for that day.
data[0]['foodbank_centre'][1]['opening_time']

[{'day': 'monday',
  'foodbank_status': 'open',
  'opening_time': '12:00',
  'closing_time': '13:00'},
 {'day': 'tuesday',
  'foodbank_status': 'open',
  'opening_time': '10:00',
  'closing_time': '12:00'},
 {'day': 'friday',
  'foodbank_status': 'open',
  'opening_time': '12:00',
  'closing_time': '13:00'}]

In [6]:
# By looping through, it is possible to pair up opening and closing times, priming the data for a time difference calculation.
for dictionary in data:
    if dictionary['foodbank_centre'] != False:
        for fbank in dictionary['foodbank_centre']:
            try:
                s=0
                for i in fbank['opening_time']:
                    print(datetime.strptime(i['opening_time'], '%H:%M'))
                    print(datetime.strptime(i['closing_time'], '%H:%M'))
                    print('difference:',(datetime.strptime(i['closing_time'], '%H:%M')-datetime.strptime(i['opening_time'], '%H:%M')).total_seconds())
                    s += (datetime.strptime(i['closing_time'], '%H:%M')-datetime.strptime(i['opening_time'], '%H:%M')).total_seconds()
                print('total differences:',s)
                print('\n')
            except KeyError:
                continue
    else:
        continue
    break # just an example, we don't need the full list

1900-01-01 13:00:00
1900-01-01 15:00:00
difference: 7200.0
total differences: 7200.0


1900-01-01 12:00:00
1900-01-01 13:00:00
difference: 3600.0
1900-01-01 10:00:00
1900-01-01 12:00:00
difference: 7200.0
1900-01-01 12:00:00
1900-01-01 13:00:00
difference: 3600.0
total differences: 14400.0


1900-01-01 14:00:00
1900-01-01 15:30:00
difference: 5400.0
total differences: 5400.0


1900-01-01 13:00:00
1900-01-01 14:30:00
difference: 5400.0
1900-01-01 13:00:00
1900-01-01 14:30:00
difference: 5400.0
total differences: 10800.0




In [7]:
fbanklist=[]
for dictionary in data:
    if dictionary['foodbank_centre'] != False:
        for fbank in dictionary['foodbank_centre']:
            try: # KeyError occurs because some dictionaries are empty
                fbank['website'] = dictionary['foodbank_information']['website']
                fbank['lat'] = fbank['centre_geolocation']['lat']
                fbank['lng'] = fbank['centre_geolocation']['lng']
                s=0
                for i in fbank['opening_time']:
                    s += (datetime.strptime(i['closing_time'], '%H:%M') -
                          datetime.strptime(i['opening_time'], '%H:%M')).total_seconds()
                fbank['total_time_open'] = s   
                fbanklist.append(fbank)
            except KeyError:
                continue
    else:
        continue

In [8]:
foodbanks = pd.DataFrame(fbanklist)
foodbanks.head()

,foodbank_name,foodbank_telephone_number,opening_time,centre_address,post_code,centre_geolocation,website,lat,lng,total_time_open,local_transport_links
0,Bon Accord Free Church,01224651000,"[{'day': 'wednesday', 'foodbank_status': 'open...","Rosemount Viaduct,<br />\r\nAberdeen<br />\r\n",AB25 1NS,"{'address': 'Rosemount Viaduct Aberdeen, AB25 ...",http://aberdeennorth.foodbank.org.uk/,57.14832,-2.109406799999988,7200.0,NaN
1,King's Community Church,01224651000,"[{'day': 'monday', 'foodbank_status': 'open', ...","10 Urquhart Road,<br />\r\nAberdeen,<br />\r\n",AB24 5LL,"{'address': '10 Urquhart Road, Aberdeen AB24 5...",http://aberdeennorth.foodbank.org.uk/,57.15445949999999,-2.0932259999999587,14400.0,NaN
2,The Mission,01224651000,"[{'day': 'thursday', 'foodbank_status': 'open'...","St Machar Dr, <br />\r\nAberdeen<br />\r\n",AB24 3RX,"{'address': 'St Machar Dr, Aberdeen, AB24 3R...",http://aberdeennorth.foodbank.org.uk/,57.1668511,-2.099606699999981,5400.0,NaN
3,Tillydrone Campus Foodstore,01224 489546,"[{'day': 'monday', 'foodbank_status': 'open', ...",Tillydrone Community Campus<br />\r\nHayton Ro...,AB24 2UY,"{'address': '40 Hayton Rd, Aberdeen AB24 2TR, ...",http://aberdeennorth.foodbank.org.uk/,57.171687435559846,-2.1160412023772324,10800.0,NaN
4,Fraserburgh Foodbank Centre *Deliveries Only*,NaN,"[{'day': 'monday', 'foodbank_status': 'closed'...",NaN,AB43 9HP,"{'address': '58 High St, Fraserburgh AB43 9HP,...",http://aberdeenshirenorth.foodbank.org.uk/,57.69403543703514,-2.007461452114878,0.0,NaN


There are two address strings for each row in the dataframe, one of which still has \r, \n and <br \> elements.

In [9]:
# Regional stats for Trussell Trust food banks. 6 Tables corresponding to 6 years.
reg_stats = pd.read_html('https://www.trusselltrust.org/news-and-blog/latest-stats/end-year-stats')

In [10]:
reg_stats[0]

,REGION,ADULTS,CHILDREN,TOTAL
0,Scotland,141195,69410,210605
1,Wales,72580,40793,113373
2,Northern Ireland,21592,15191,36783
3,North East,55713,32995,88708
4,North West,139703,83019,222722
5,Yorkshire & Humberside,57252,32589,89841
6,West Midlands,91212,51022,142234
7,East Midlands,47331,28328,75659
8,East,96037,60044,156081
9,London,106737,59775,166512


Now we have regional statistics and a postcode associated with every food bank. We could use data relating postcodes to regions.

In [11]:
codes_regions = pd.read_html('https://www.robertsharp.co.uk/2017/08/09/a-table-that-shows-the-uk-region-for-all-postcode-districts/')[0]

In [12]:
codes_regions[['UK region', 'Postcode prefix']]

,UK region,Postcode prefix
0,Scotland,AB
1,East of England,AL
2,West Midlands,B
3,South West,BA
4,North West,BB
...,...,...
124,West Midlands,WS
125,West Midlands,WV
126,Non-geographic,XX
127,North East,YO
